## Trabalhando com o comando SELECT em Dataframes
Neste notebook iremos utilizar o comando SELECT em nossos Dataframes do Spark. <br>Cobriremos apenas a sintaxe básica para que você aprenda a converter suas consultas básicas SQL em Spark Dataframes!<br>
Caso não conheça SQL, não há nenhum problema :)

### Importando as bibliotecas
Nesta etapa iremos apenas importar todas as bibliotecas e funções necessárias para rodar o programa

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc, col, upper, lower

### Criando uma SparkSession
Por meio de uma SparkSession terei acesso ao SparkContext da minha aplicação.

In [5]:
spark = SparkSession.builder.appName('Select').getOrCreate()

### Carregando os dados
Nesta etapa estamos carregando os dados que utilizaremos neste notebook

In [7]:
dados = spark \
    .read.option("header","true") \
    .option("inferSchema","true") \
    .option("delimiter",";") \
    .format("csv") \
    .load("/FileStore/tables/bank_additional_full-3fd09.csv")

### Filtrando e ordenando dados (consultas básicas)
Agora sim, finalmente faremos nossos primeiras consultas utilizando o comando SELECT em Dataframes.<br>
Vamos começar com dois tipos de operações muito úteis no dia a dia: filtros e ordenação

**Seleciona apenas os primeiros registros da coluna "age"**

In [10]:
dados.select(['age', 'job']).filter(dados['age'] > 55).orderBy(desc('age')).show(2)

+---+-------+
age| job|
+---+-------+
 98|retired|
 98|retired|
+---+-------+
only showing top 2 rows

**Filtra por estado civil casado**

In [12]:
dados.select(['age', 'marital']).filter(dados['marital'] == 'married').show(5)

+---+-------+
age|marital|
+---+-------+
 56|married|
 57|married|
 37|married|
 40|married|
 56|married|
+---+-------+
only showing top 5 rows

**Filtra por estado civil diferente de casado**

In [14]:
dados.select(['age', 'marital']).where(dados['marital'] != 'married').show(5) 
dados.select(['age', 'marital']).where("marital != 'married' ").show(5)

+---+--------+
age| marital|
+---+--------+
 24| single|
 25| single|
 25| single|
 29| single|
 57|divorced|
+---+--------+
only showing top 5 rows

+---+--------+
age| marital|
+---+--------+
 24| single|
 25| single|
 25| single|
 29| single|
 57|divorced|
+---+--------+
only showing top 5 rows

**Mostrando valores únicos**

In [16]:
dados.select('marital').distinct().show()

+--------+
 marital|
+--------+
 unknown|
divorced|
 married|
 single|
+--------+

**Especificando vários filtros em comando separados**

In [18]:
filtro_idade = dados['age'] > 40
filtro_civil = dados['marital'].contains("married")
dados.select(['age', 'marital', 'job']).where(dados['job'].isin("unemployed", "retired")).where(filtro_civil & filtro_idade).show(5)

+---+-------+----------+
age|marital| job|
+---+-------+----------+
 54|married| retired|
 55|married|unemployed|
 58|married|unemployed|
 56|married| retired|
 54|married| retired|
+---+-------+----------+
only showing top 5 rows

**Note que os filtros também poderiam ser usados no comando select**

In [20]:
dados.select(['age', 'marital', 'job']).withColumn("filtro_civil", filtro_civil).show(5)

+---+-------+---------+------------+
age|marital| job|filtro_civil|
+---+-------+---------+------------+
 56|married|housemaid| true|
 57|married| services| true|
 37|married| services| true|
 40|married| admin.| true|
 56|married| services| true|
+---+-------+---------+------------+
only showing top 5 rows

### Convertendo dados
Outra tarefa bastante comum para manipular dados refere-se a conversão de tipos de dados

In [22]:
dados1 = dados.withColumn("idade_string", dados['age'].cast("string"))
dados1.select('idade_string')

Out[50]: DataFrame[idade_string: string]

### Trabalhando com funções do Spark
Concluindo este notebook, também é útil saber que existem funções do próprio spark que podemos aplicar em nossos dados.<br>
Vou demonstrar apenas um exemplo abaixo, mas sugiro que você acesse a documentação do Spark para ter uma idéia das funções existentes!

In [24]:
dados.select(upper(dados['poutcome'])).show(1)

+---------------+
upper(poutcome)|
+---------------+
 NONEXISTENT|
+---------------+
only showing top 1 row

In [25]:
dados.select(lower(dados['poutcome'])).show(1)

+---------------+
lower(poutcome)|
+---------------+
 nonexistent|
+---------------+
only showing top 1 row

### Obrigado!
Quer construir uma carreira em Data Science? Acesse meu blog pessoal em https://www.hackinganalytics.com/